# Data Cleaning & Feature Engineering

This notebook transforms the raw scraped data into a clean, model-ready dataset through systematic cleaning and feature engineering.

## Load Data

In [1]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv('../data/02_intermediate/house_data_with_city.csv')
print(f"Initial dataset: {df.shape[0]} rows, {df.shape[1]} columns")
df.head()

Initial dataset: 949 rows, 9 columns


,URL,Price,Address,Bedrooms,Bathrooms,House_Size,Land_Size,Description,City
0,https://ikman.lk/en/ad/swimming-pool-with-luxu...,69000000,Kesbewa,5,5,"2,800.0 sqft",20.0 perches,✳️ Brand New Super Luxury Modern Villa’s For S...,Kesbewa
1,https://ikman.lk/en/ad/modern-house-for-sale-i...,200000000,NaN,9,8,"9,000.0 sqft",20.0 perches,luxury New House for sale in Thalawathugoda€12...,Thalawathugoda
2,https://ikman.lk/en/ad/luxury-house-for-sale-i...,39000000,Kesbewa,4,4,"2,680.0 sqft",6.2 perches,"✅ පිලියන්දල ටවුමට නුදුරුව, කැස්බැව හංදියට ඇවිද...",Kesbewa
3,https://ikman.lk/en/ad/kaduwela-nawagamuwa-two...,19000000,Kaduwela Nawagamuwa,4,2,"2,280.0 sqft",10.5 perches,TWO STORY HOUSE FOR SALE IN KADUWELA NAWAGAMU...,Kaduwela
4,https://ikman.lk/en/ad/single-storied-best-hou...,22000000,"Galwarusawa Rd, Athurugiriya",3,2,"1,366.0 sqft",8.15 perches,Single Storied House For Sale Galwarusawa Roa...,Athurugiriya


## Step 1: Clean House_Size & Land_Size

Strip units, commas, and convert to numeric values for model compatibility.

In [2]:
def clean_size(value):
    """Extract numeric value from size strings"""
    if pd.isna(value):
        return np.nan
    value_str = str(value)
    value_str = re.sub(r'[^\d.]', '', value_str)
    try:
        return float(value_str) if value_str else np.nan
    except ValueError:
        return np.nan

df['House_Size'] = df['House_Size'].apply(clean_size)
df['Land_Size'] = df['Land_Size'].apply(clean_size)

print(f"House_Size - Non-null: {df['House_Size'].notna().sum()}, Mean: {df['House_Size'].mean():.2f}")
print(f"Land_Size - Non-null: {df['Land_Size'].notna().sum()}, Mean: {df['Land_Size'].mean():.2f}")

House_Size - Non-null: 949, Mean: 3027.20
Land_Size - Non-null: 949, Mean: 12.58


## Step 2: Logic Check & Filtering

Remove invalid entries to prevent model confusion.

In [5]:
initial_count = len(df)

df['Bedrooms'] = pd.to_numeric(df['Bedrooms'], errors='coerce')
df['Bathrooms'] = pd.to_numeric(df['Bathrooms'], errors='coerce')
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')

df = df[df['Bedrooms'] > 0]
df = df[(df['Price'] >= 1_000_000) & (df['Price'] <= 2_000_000_000)]

removed_count = initial_count - len(df)
print(f"Removed {removed_count} invalid rows ({removed_count/initial_count*100:.1f}%)")
print(f"Remaining: {len(df)} rows")
print(f"Price range: LKR {df['Price'].min():,.0f} - LKR {df['Price'].max():,.0f}")

Removed 0 invalid rows (0.0%)
Remaining: 946 rows
Price range: LKR 1,500,000 - LKR 950,000,000


## Step 3: Feature Engineering - Text Extraction

Extract valuable signals from property descriptions.

In [6]:
def extract_binary_features(description):
    """Extract key property attributes from text"""
    if pd.isna(description):
        return 0, 0, 0
    
    desc_lower = str(description).lower()
    
    is_brand_new = 1 if any(keyword in desc_lower for keyword in ['brand new', 'newly built', 'new house']) else 0
    is_modern = 1 if any(keyword in desc_lower for keyword in ['modern', 'luxury', 'contemporary']) else 0
    has_parking = 1 if any(keyword in desc_lower for keyword in ['parking', 'garage', 'car park']) else 0
    
    return is_brand_new, is_modern, has_parking

df[['Is_Brand_New', 'Is_Modern', 'Has_Parking']] = df['Description'].apply(
    lambda x: pd.Series(extract_binary_features(x))
)

print(f"Brand New: {df['Is_Brand_New'].sum()} ({df['Is_Brand_New'].mean()*100:.1f}%)")
print(f"Modern/Luxury: {df['Is_Modern'].sum()} ({df['Is_Modern'].mean()*100:.1f}%)")
print(f"Parking: {df['Has_Parking'].sum()} ({df['Has_Parking'].mean()*100:.1f}%)")

Brand New: 183 (19.3%)
Modern/Luxury: 395 (41.8%)
Parking: 121 (12.8%)


## Step 4: Feature Engineering - City Tiering

Group cities by median price to solve the high-cardinality problem.

In [7]:
city_median_prices = df.groupby('City')['Price'].median().sort_values(ascending=False)
print(f"Total unique cities: {len(city_median_prices)}")
print(f"\nTop 10 most expensive cities:")
print(city_median_prices.head(10))

Total unique cities: 57

Top 10 most expensive cities:
City
Colombo 4     300000000.0
Colombo 7     235000000.0
Colombo 1     169000000.0
Nawala        133000000.0
Kohuwala      132500000.0
Kotte         130500000.0
Dehiwala      120000000.0
Kalubowila     97500000.0
Colombo 3      97000000.0
Colombo 2      95500000.0
Name: Price, dtype: float64


In [8]:
n_tiers = 6
city_median_prices = df.groupby('City')['Price'].median().sort_values(ascending=False)

tier_labels = {1: 'Luxury', 2: 'Premium', 3: 'Upper-Mid', 4: 'Mid-Range', 5: 'Affordable', 6: 'Budget'}
city_tiers = pd.qcut(city_median_prices, q=n_tiers, labels=range(1, n_tiers+1), duplicates='drop')
city_tier_map = city_tiers.to_dict()

df['City_Tier'] = df['City'].map(city_tier_map)

print(f"City distribution across tiers:")
for tier in range(1, n_tiers+1):
    tier_cities = [city for city, t in city_tier_map.items() if t == tier]
    tier_count = (df['City_Tier'] == tier).sum()
    print(f"  Tier {tier} ({tier_labels[tier]}): {len(tier_cities)} cities, {tier_count} properties")

City distribution across tiers:
  Tier 1 (Luxury): 10 cities, 70 properties
  Tier 2 (Premium): 9 cities, 199 properties
  Tier 3 (Upper-Mid): 10 cities, 234 properties
  Tier 4 (Mid-Range): 10 cities, 166 properties
  Tier 5 (Affordable): 8 cities, 179 properties
  Tier 6 (Budget): 10 cities, 98 properties


## Step 5: Encoding

Apply label encoding to preserve ordinal relationship in city tiers.

In [9]:
df['City_Tier'] = df['City_Tier'].astype(int)

print(f"City_Tier encoding complete")
print(f"Value range: {df['City_Tier'].min()} to {df['City_Tier'].max()}")
print(f"\nCity Tier vs Median Price:")
print(df.groupby('City_Tier')['Price'].median().sort_index())

City_Tier encoding complete
Value range: 1 to 6

City Tier vs Median Price:
City_Tier
1     19500000.0
2     29500000.0
3     39000000.0
4     64500000.0
5     79000000.0
6    129750000.0
Name: Price, dtype: float64


## Step 6: Scaling & Normalization

Transform distributions for improved model performance.

In [10]:
from sklearn.preprocessing import StandardScaler

df['Price_Log'] = np.log(df['Price'])

scaler = StandardScaler()
df[['House_Size_Scaled', 'Land_Size_Scaled']] = scaler.fit_transform(
    df[['House_Size', 'Land_Size']].fillna(df[['House_Size', 'Land_Size']].median())
)

print(f"Price distribution (original):")
print(f"  Mean: LKR {df['Price'].mean():,.0f}, Std: LKR {df['Price'].std():,.0f}")
print(f"  Skewness: {df['Price'].skew():.2f}")

print(f"\nPrice distribution (log-transformed):")
print(f"  Mean: {df['Price_Log'].mean():.2f}, Std: {df['Price_Log'].std():.2f}")
print(f"  Skewness: {df['Price_Log'].skew():.2f}")

print(f"\nHouse_Size scaled: Mean={df['House_Size_Scaled'].mean():.2f}, Std={df['House_Size_Scaled'].std():.2f}")
print(f"Land_Size scaled: Mean={df['Land_Size_Scaled'].mean():.2f}, Std={df['Land_Size_Scaled'].std():.2f}")

Price distribution (original):
  Mean: LKR 69,967,125, Std: LKR 79,397,769
  Skewness: 4.78

Price distribution (log-transformed):
  Mean: 17.74, Std: 0.74
  Skewness: 0.57

House_Size scaled: Mean=0.00, Std=1.00
Land_Size scaled: Mean=0.00, Std=1.00


## Step 7: Cleanup

Drop columns that have been transformed or are no longer needed.

In [11]:
columns_to_drop = ['URL', 'Address', 'Description', 'City', 'House_Size', 'Land_Size', 'Price']
df_clean = df.drop(columns=columns_to_drop)

print(f"Final dataset shape: {df_clean.shape[0]} rows, {df_clean.shape[1]} columns")
print(f"\nFinal columns:")
for col in df_clean.columns:
    print(f"  - {col}")
    
df_clean.head()

Final dataset shape: 946 rows, 9 columns

Final columns:
  - Bedrooms
  - Bathrooms
  - Is_Brand_New
  - Is_Modern
  - Has_Parking
  - City_Tier
  - Price_Log
  - House_Size_Scaled
  - Land_Size_Scaled


,Bedrooms,Bathrooms,Is_Brand_New,Is_Modern,Has_Parking,City_Tier,Price_Log,House_Size_Scaled,Land_Size_Scaled
0,5.0,5.0,1,1,0,3,18.049617,-0.106345,0.656478
1,9.0,8.0,1,1,0,5,19.113828,2.946412,0.656478
2,4.0,4.0,0,0,0,3,17.479072,-0.165431,-0.562943
3,4.0,2.0,0,0,1,1,16.759950,-0.362383,-0.182978
4,3.0,2.0,0,0,0,2,16.906553,-0.812418,-0.390633


## Save Processed Data

In [12]:
output_path = '../data/03_processed/cleaned_house_data.csv'
df_clean.to_csv(output_path, index=False)

print(f"✓ Cleaned data saved to: {output_path}")
print(f"✓ Final dataset: {df_clean.shape[0]} rows, {df_clean.shape[1]} features")
print(f"✓ Target variable: Price_Log (log-transformed price)")

✓ Cleaned data saved to: ../data/03_processed/cleaned_house_data.csv
✓ Final dataset: 946 rows, 9 features
✓ Target variable: Price_Log (log-transformed price)


## Summary

**Data Quality Improvements:**
- Cleaned numeric fields (House_Size, Land_Size)
- Removed invalid entries (0 bedrooms, outlier prices)
- Extracted 3 binary features from descriptions

**Feature Engineering:**
- Created City_Tier to reduce cardinality from {len_cities} → 6 tiers
- Log-transformed Price to normalize distribution
- Scaled House_Size and Land_Size to standardized ranges

**Ready for Modeling:** {final_rows} clean samples with {final_cols} features